# Capstone

## Part 1 - Analytics on Raw Data

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
all_files = glob.glob("Raw_Data/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files))

**Basic checks have been done on the data to make sure everything is generally ok; nothing has been missed**

In [3]:
# df.head()
# df[['YEAR', 'MONTH']].drop_duplicates().sort_values(['YEAR', 'MONTH'])

**We will also do some data cleanup in this section:**
* We will filter to only the columns that are of interest to us for this Capstone project
* Only Ancorage origin airport will be looked at
* The actual carrier descriptions will be shown
* Cancellation code descriptions will be shown
* Delays over 60 minutes will be looked at - a respective column will be added to the data

In [4]:
# Columns that will be kept for analysis
columns_to_keep =  [
    "YEAR",
    "QUARTER",
    "MONTH",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_UNIQUE_CARRIER",
    "TAIL_NUM",
    "OP_CARRIER_FL_NUM",
    "ORIGIN",
    "DEST",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "CANCELLED",
    "CANCELLATION_CODE",
    "DIVERTED",
    "AIR_TIME",
    "DISTANCE",
    "CARRIER_DELAY",
    "WEATHER_DELAY",
    "NAS_DELAY",
    "SECURITY_DELAY",
    "LATE_AIRCRAFT_DELAY" ]

# Delay threshold of 60 mins
delay_threshold = 60

# Filtering & Replacing
df1 = df.copy()
df1 = df1[columns_to_keep]
df1 = df1[df1['ORIGIN'] == 'ANC']
df1['OP_UNIQUE_CARRIER'] = df1['OP_UNIQUE_CARRIER'].replace(['AS', 'DL', 'UA', 'AA'], ['Alaska Airlines', 'Delta', 'United', 'American Airlines'])
df1['CANCELLATION_CODE'] = df1['CANCELLATION_CODE'].replace(['A', 'B', 'C'], ['Carrier', 'Weather', 'National Air System'])

# Create totals & classification of the delay
df1['TOTAL_DELAY'] = df1["CARRIER_DELAY"] + df1["WEATHER_DELAY"] + df1["NAS_DELAY"] + df1["SECURITY_DELAY"] + df1["LATE_AIRCRAFT_DELAY"]
df1['CRITICAL_DELAY'] = np.where(
    (df1['TOTAL_DELAY'] >= delay_threshold) | 
    (df1['CANCELLED'] == 1) | 
    (df1['DIVERTED'] == 1),
    1,  # if true
    0   # else
)

In [5]:
df1.head()

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK  OP_UNIQUE_CARRIER  \
0   2023        1      1             1            7  American Airlines   
3   2023        1      1             1            7    Alaska Airlines   
7   2023        1      1             1            7    Alaska Airlines   
8   2023        1      1             1            7    Alaska Airlines   
10  2023        1      1             1            7    Alaska Airlines   

   TAIL_NUM  OP_CARRIER_FL_NUM ORIGIN DEST  ...  DIVERTED  AIR_TIME  DISTANCE  \
0    N434AN                399    ANC  DFW  ...       0.0     381.0    3043.0   
3    N215AK                 92    ANC  SEA  ...       0.0     196.0    1448.0   
7    N265AK                573    ANC  SEA  ...       0.0     194.0    1448.0   
8    N267AK                170    ANC  FAI  ...       0.0      39.0     261.0   
10   N267AK                722    ANC  ORD  ...       0.0     345.0    2846.0   

   CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  SECURITY_DELAY  \
0            NaN            NaN        NaN             NaN   
3            0.0            0.0       18.0             0.0   
7            0.0            0.0       16.0             0.0   
8            NaN            NaN        NaN             NaN   
10          18.0            0.0       20.0             0.0   

    LATE_AIRCRAFT_DELAY  TOTAL_DELAY  CRITICAL_DELAY  
0                   NaN          NaN               0  
3                   0.0         18.0               0  
7                   0.0         16.0               0  
8                   NaN          NaN               0  
10                  0.0         38.0               0  

[5 rows x 24 columns]

In [6]:
#df1['OP_UNIQUE_CARRIER'].value_counts()
#df1['CANCELLATION_CODE'].value_counts()
#df1['CANCELLED'].value_counts()
#df1['DIVERTED'].value_counts()
df1['CRITICAL_DELAY'].value_counts(normalize = True)

CRITICAL_DELAY
0    0.943643
1    0.056357
Name: proportion, dtype: float64